# ADA - Banco de Dados I - Projeto Final

## Professor: Tiago Dias

## Grupo 1:

* Alana Nunes
* Deivid Gabriel
* Ewerton Costa
* Hugo Negrão
* Vinicius Silva

### TEMA: Consumo de API pelo python com integração a um Banco de Dados


**Contexto**

A API de Star Wars, ou "swapi" (Swah-pee) é a primeira fonte de dados quantificada e acessível por meio de programação para todos os dados do universo canônico de Star Wars!

esses dados foram obtidos de https://swapi.dev/ e são apenas para fins educativos.


Está API contém os Dados de Star Wars: Planetas, Naves Espaciais, Veículos, Pessoas, Filmes e Espécies de todos os SETE filmes de Star Wars.


**Proposta**

Este projeto tem por finalidade o consumo de uma API através do python e a construção de um Bando de Dados com os conhecimentos obtidos no módulo DS-PY-006 BANCO DE DADOS I - Turma 889 - Diversidade Tech - Suzano.

In [5]:
# Bibliotecas Utilizadas
import numpy as np
import pandas as pd
import requests
import psycopg2
import pandas.io.sql as psql
import time

### Conexão do Python com a API

In [6]:
def get_dados_semi_completos_in_df(tipo: str) -> pd.DataFrame:
    '''
    Extrai dados via The Star Wars API, limitado a 10 linhas.

    Parâmetros:
        tipo (str): tipo de dado a ser extraído - planets, spaceships, people, vehicles, films ou species

    Retorna:
        df (dataframe): dado extraído em formato de pandas dataframe
    '''
    url = f'https://swapi.dev/api/{tipo}/'
    resposta = requests.get(url)
    dados = resposta.json()
    print(resposta.status_code)
    df = pd.DataFrame.from_dict(dados.get('results'))
    return df


def get_dados_completos_in_df(tipo):
    '''
    Extrai dados via The Star Wars API, pegando todos os registros.
    
    Parâmetros:
        tipo (str): tipo de dado a ser extraído - planets, starships, people, vehicles, films ou species
        
    Retorna:
        df (dataframe): dado extraído em formato de pandas dataframe
    '''
    df = pd.DataFrame()
    ids_sem_dados = []
    
    for i in range(1,91):
        try:
            url = f'https://swapi.dev/api/{tipo}/{i}/'
            resposta = requests.get(url)
            dados = resposta.json()

            if resposta.status_code == 200:
                df1 = pd.DataFrame.from_dict(dados, orient ='index')
                df1 = df1.transpose()
                df1[f'id_{tipo}'] = i                  
                df = pd.concat([df, df1])
                time.sleep(1)
            else:
                ids_sem_dados.append(i)
                time.sleep(1)
                pass
        except:
            print(f'Base {tipo}: Erro na requisição - {i}')
            time.sleep(1)
    print(f'Para a base {tipo}, as seguintes requisições estavam vazias: \n{ids_sem_dados}')
    return df


def tratar_df(df: pd.DataFrame, tipo) -> pd.DataFrame:
    '''
    Trata o dataframe para manipulação.

    Parâmetros:
        df (dataframe): dataframe a ser tratado
        tipo (str): tipo de dado do dataframe - planets, starships, people

    Retorna:
        df (dataframe): dataframe tratado
    '''
    df = df.replace('unknown', np.nan)
    df = df.replace('n/a', np.nan)
    df = df.replace('N/A', np.nan)
    df = df.fillna(0)
    if tipo == 'planets':
        df['name'] = df['name'].replace(0, 'unknown')
    return df

def selecionar_colunas(df: pd.DataFrame, tipo: str) -> pd.DataFrame:
    '''
    Seleciona apenas as colunas que serão usadas para popular o banco de dados

    Parâmetros:
        df (dataframe): dataframe a ser tratado
        tipo (str): tipo de dado do dataframe - planets, starships, people

    Retorna:
        df (dataframe): dataframe tratado
    '''
    if tipo == 'planets':
        new_cols = ['id_planets', 'name', 'diameter', 'climate', 'gravity', 'terrain', 'population']
        df = df[new_cols]
        return df
        
    elif tipo == 'people':
        new_cols = ['id_people', 'name', 'height', 'mass', 'hair_color', 'skin_color', 'birth_year', 'gender', 'homeworld']
        df = df[new_cols]
        df.rename(columns = {'homeworld':'id_planets'}, inplace = True)
        return df

    elif tipo == 'starships':
        new_cols = ['id_starships', 'name', 'model', 'manufacturer', 'cost_in_credits', 'length', 'crew', 'passengers', 'consumables', 'pilots']
        df = df[new_cols]
        df.rename(columns = {'pilots':'id_people'}, inplace = True)
        return df

def save_csv(df: pd.DataFrame, tipo: str, raw: str):
    '''
    Salva dataframe em formato csv.
    '''
    if raw == 'yes':
        df.to_csv(f'{tipo}_raw.csv', sep=';', encoding='utf-8', index=False)
    else:
        df.to_csv(f'{tipo}.csv', sep=';', encoding='utf-8', index=False)

def tratar_coluna_chave_estrangeira(row, coluna: str):
    '''
    Tratar coluna que vem em formato de url para o formato de id. A função será utilizada com o método apply.
    
    Parâmetros:
        coluna (str): coluna a ser tratada

    Retorna:
        row[coluna] (series): coluna tratada
    '''
    if coluna == 'id_planets':
        row[coluna] = row[coluna][-3:-1]
        if '/' in row[coluna]:
            row[coluna] = row[coluna][-1]
        return row[coluna]

    elif coluna == 'id_people':
        if row[coluna] == '[]':
            row[coluna] = np.nan
        else:
            row[coluna] = row[coluna][31:33]
            if '/' in row[coluna]:
                row[coluna] = row[coluna][-2]
        return row[coluna]
    
def adicionar_linha_nula_id0(df: pd.DataFrame, tipo: str) -> pd.DataFrame:
    '''
    Adiciona uma linha nula com id_people = 0 se dataframe people ou id_planets = 0 se dataframe planets

    Parâmetros:
        df (dataframe): dataframe a ser tratado
        tipo (str): tipo de dado do dataframe - planets, starships, people

    Retorna:
        df (dataframe): dataframe tratado
    '''
    if tipo == 'people':
        dict_people_id0 = {
            'id_people': 0,
            'name': np.nan,
            'height': 0,
            'mass': 0,
            'hair_color': np.nan, 
            'skin_color': np.nan,
            'birth_year':np.nan, 
            'gender':np.nan, 
            'id_planets': 0
        }
        df.loc[len(df)+1] = dict_people_id0
        return df
    
    elif tipo == 'planets':
        dict_planets_id0 = {
            'id_planets': 0,
            'name': np.nan,
            'diameter': 0,
            'climate': np.nan,
            'gravity': np.nan, 
            'terrain': np.nan,
            'population':0
        }
        df.loc[len(df)+1] = dict_planets_id0
        return df

### Aquisição dos dados

In [3]:
%%time
# Para recuperar os dados semi_completos (esse link só retorna 10 linhas), no exemplo, para dados de people (pessoas)
df = get_dados_semi_completos_in_df(tipo = 'people')
df.head(1)

200
Wall time: 3.86 s


,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],"[https://swapi.dev/api/vehicles/14/, https://s...","[https://swapi.dev/api/starships/12/, https://...",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,https://swapi.dev/api/people/1/


In [4]:
# Recuperar os dados completos de people (pessoas)
df_people_raw = get_dados_completos_in_df(tipo = 'people')

Para a base people, as seguintes requisições estavam vazias: 
[17, 84, 85, 86, 87, 88, 89, 90]


In [7]:
# Recuperar os dados completos de starships (naves)
df_starships_raw = get_dados_completos_in_df(tipo = 'starships')

Para a base starships, as seguintes requisições estavam vazias: 
[1, 4, 6, 7, 8, 14, 16, 18, 19, 20, 24, 25, 26, 30, 33, 34, 35, 36, 37, 38, 42, 44, 45, 46, 50, 51, 53, 54, 55, 56, 57, 60, 62, 67, 69, 70, 71, 72, 73, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]


In [6]:
# Recuperar os dados completos de planets (planetas)
df_planets_raw = get_dados_completos_in_df(tipo = 'planets')

Para a base planets, as seguintes requisições estavam vazias: 
[61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]


In [8]:
# Verificar as informações do dataframe
df_planets_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 0
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             60 non-null     object
 1   rotation_period  60 non-null     object
 2   orbital_period   60 non-null     object
 3   diameter         60 non-null     object
 4   climate          60 non-null     object
 5   gravity          60 non-null     object
 6   terrain          60 non-null     object
 7   surface_water    60 non-null     object
 8   population       60 non-null     object
 9   residents        60 non-null     object
 10  films            60 non-null     object
 11  created          60 non-null     object
 12  edited           60 non-null     object
 13  url              60 non-null     object
 14  id_planets       60 non-null     int64 
dtypes: int64(1), object(14)
memory usage: 7.5+ KB


In [9]:
df_planets_raw.head()

,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,residents,films,created,edited,url,id_planets
0,Tatooine,23,304,10465,arid,1 standard,desert,1,200000,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-09T13:50:49.641000Z,2014-12-20T20:58:18.411000Z,https://swapi.dev/api/planets/1/,1
0,Alderaan,24,364,12500,temperate,1 standard,"grasslands, mountains",40,2000000000,"[https://swapi.dev/api/people/5/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T11:35:48.479000Z,2014-12-20T20:58:18.420000Z,https://swapi.dev/api/planets/2/,2
0,Yavin IV,24,4818,10200,"temperate, tropical",1 standard,"jungle, rainforests",8,1000,[],[https://swapi.dev/api/films/1/],2014-12-10T11:37:19.144000Z,2014-12-20T20:58:18.421000Z,https://swapi.dev/api/planets/3/,3
0,Hoth,23,549,7200,frozen,1.1 standard,"tundra, ice caves, mountain ranges",100,unknown,[],[https://swapi.dev/api/films/2/],2014-12-10T11:39:13.934000Z,2014-12-20T20:58:18.423000Z,https://swapi.dev/api/planets/4/,4
0,Dagobah,23,341,8900,murky,N/A,"swamp, jungles",8,unknown,[],"[https://swapi.dev/api/films/2/, https://swapi...",2014-12-10T11:42:22.590000Z,2014-12-20T20:58:18.425000Z,https://swapi.dev/api/planets/5/,5


In [10]:
# Verificar as informações do dataframe
df_people_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82 entries, 0 to 0
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        82 non-null     object
 1   height      82 non-null     object
 2   mass        82 non-null     object
 3   hair_color  82 non-null     object
 4   skin_color  82 non-null     object
 5   eye_color   82 non-null     object
 6   birth_year  82 non-null     object
 7   gender      82 non-null     object
 8   homeworld   82 non-null     object
 9   films       82 non-null     object
 10  species     82 non-null     object
 11  vehicles    82 non-null     object
 12  starships   82 non-null     object
 13  created     82 non-null     object
 14  edited      82 non-null     object
 15  url         82 non-null     object
 16  id_people   82 non-null     int64 
dtypes: int64(1), object(16)
memory usage: 11.5+ KB


In [11]:
df_people_raw.head()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url,id_people
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],"[https://swapi.dev/api/vehicles/14/, https://s...","[https://swapi.dev/api/starships/12/, https://...",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,https://swapi.dev/api/people/1/,1
0,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,https://swapi.dev/api/people/2/,2
0,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,https://swapi.dev/api/planets/8/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:11:50.376000Z,2014-12-20T21:17:50.311000Z,https://swapi.dev/api/people/3/,3
0,Darth Vader,202,136,none,white,yellow,41.9BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[https://swapi.dev/api/starships/13/],2014-12-10T15:18:20.704000Z,2014-12-20T21:17:50.313000Z,https://swapi.dev/api/people/4/,4
0,Leia Organa,150,49,brown,light,brown,19BBY,female,https://swapi.dev/api/planets/2/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[https://swapi.dev/api/vehicles/30/],[],2014-12-10T15:20:09.791000Z,2014-12-20T21:17:50.315000Z,https://swapi.dev/api/people/5/,5


In [12]:
# Verificar as informações do dataframe
df_starships_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 0 to 0
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    36 non-null     object
 1   model                   36 non-null     object
 2   manufacturer            36 non-null     object
 3   cost_in_credits         36 non-null     object
 4   length                  36 non-null     object
 5   max_atmosphering_speed  36 non-null     object
 6   crew                    36 non-null     object
 7   passengers              36 non-null     object
 8   cargo_capacity          36 non-null     object
 9   consumables             36 non-null     object
 10  hyperdrive_rating       36 non-null     object
 11  MGLT                    36 non-null     object
 12  starship_class          36 non-null     object
 13  pilots                  36 non-null     object
 14  films                   36 non-null     object
 15  created  

In [13]:
df_starships_raw.head()

,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilots,films,created,edited,url,id_starships
0,CR90 corvette,CR90 corvette,Corellian Engineering Corporation,3500000,150,950,30-165,600,3000000,1 year,2.0,60,corvette,[],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T14:20:33.369000Z,2014-12-20T21:23:49.867000Z,https://swapi.dev/api/starships/2/,2
0,Star Destroyer,Imperial I-class Star Destroyer,Kuat Drive Yards,150000000,"1,600",975,"47,060",n/a,36000000,2 years,2.0,60,Star Destroyer,[],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T15:08:19.848000Z,2014-12-20T21:23:49.870000Z,https://swapi.dev/api/starships/3/,3
0,Sentinel-class landing craft,Sentinel-class landing craft,"Sienar Fleet Systems, Cyngus Spaceworks",240000,38,1000,5,75,180000,1 month,1.0,70,landing craft,[],[https://swapi.dev/api/films/1/],2014-12-10T15:48:00.586000Z,2014-12-20T21:23:49.873000Z,https://swapi.dev/api/starships/5/,5
0,Death Star,DS-1 Orbital Battle Station,"Imperial Department of Military Research, Sien...",1000000000000,120000,n/a,"342,953","843,342",1000000000000,3 years,4.0,10,Deep Space Mobile Battlestation,[],[https://swapi.dev/api/films/1/],2014-12-10T16:36:50.509000Z,2014-12-20T21:26:24.783000Z,https://swapi.dev/api/starships/9/,9
0,Millennium Falcon,YT-1300 light freighter,Corellian Engineering Corporation,100000,34.37,1050,4,6,100000,2 months,0.5,75,Light freighter,"[https://swapi.dev/api/people/13/, https://swa...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T16:59:45.094000Z,2014-12-20T21:23:49.880000Z,https://swapi.dev/api/starships/10/,10


In [14]:
# Salvar os dataframes raw (sem tratamento) em csv
save_csv(df = df_people_raw, tipo = 'people', raw = 'yes')
save_csv(df = df_starships_raw, tipo = 'starships', raw = 'yes')
save_csv(df = df_planets_raw, tipo = 'planets', raw = 'yes')

### Tratando os dataframes

In [7]:
# Lendo os arquivos raw
df_planets = pd.read_csv('planets_raw.csv', sep=';')
df_people = pd.read_csv('people_raw.csv', sep=';')
df_starships = pd.read_csv('starships_raw.csv', sep=';')

In [8]:
#Selecionando as colunas válidas para o banco de dados
df_planets = selecionar_colunas(df = df_planets, tipo = 'planets')
df_people = selecionar_colunas(df = df_people, tipo = 'people')
df_starships = selecionar_colunas(df = df_starships, tipo = 'starships')

C:\Users\deivi\AppData\Local\Temp\ipykernel_8936\3633219664.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'homeworld':'id_planets'}, inplace = True)
C:\Users\deivi\AppData\Local\Temp\ipykernel_8936\3633219664.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'pilots':'id_people'}, inplace = True)


In [9]:
# Tratando colunas que vão servir como chave estrangeira
df_people['id_planets'] = df_people.apply(lambda row: tratar_coluna_chave_estrangeira(row, 'id_planets'), axis=1)
df_starships['id_people'] = df_starships.apply(lambda row: tratar_coluna_chave_estrangeira(row, 'id_people'), axis=1)

In [10]:
# Limpando os null, os missings e os 'n/a'
df_starships = tratar_df(df = df_starships, tipo = 'starships')
df_people = tratar_df(df = df_people, tipo = 'people')
df_planets = tratar_df(df = df_planets,  tipo = 'planets')

### Para versões 3.9 para cima

In [13]:
# Tratando dados problemáticos
df_people['mass'].replace('1,358', 1358, inplace = True)
df_starships['passengers'].replace('843,342', 843342, inplace = True)
df_starships['length'].replace('1,600', 1600, inplace = True)
df_starships['crew'].replace('30-165', 165, inplace = True)
df_starships['crew'].replace('47,060', 47060, inplace = True)
df_starships['crew'].replace('342,953', 342953, inplace = True)
df_starships['crew'].replace('279,144', 279144, inplace = True)

#Fazendo a conversão de algumas colunas restantes para int
df_planets['diameter'] = df_planets['diameter'].astype(int)
df_planets['population'] = df_planets['population'].astype(np.int64)
df_people['height'] = df_people['height'].astype(int)
df_starships['cost_in_credits'] = df_starships['cost_in_credits'].astype(np.int64)
df_starships['crew'] = df_starships['crew'].astype(int)

In [15]:
df_starships.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_starships     36 non-null     int64 
 1   name             36 non-null     object
 2   model            36 non-null     object
 3   manufacturer     36 non-null     object
 4   cost_in_credits  36 non-null     int64 
 5   length           36 non-null     object
 6   crew             36 non-null     int32 
 7   passengers       36 non-null     object
 8   consumables      36 non-null     object
 9   id_people        36 non-null     object
dtypes: int32(1), int64(2), object(7)
memory usage: 2.8+ KB


### Para versões anteriores

In [7]:
# # Tratando dados problemáticos
# df_people['mass'][df_people['mass'] == '1,358'] = 1358
# df_starships['passengers'][df_starships['passengers'] == '843,342'] = 843342
# df_starships['length'][df_starships['length'] == '1,600'] = 1600
# df_starships['crew'][df_starships['crew'] == '30-165'] = 165
# df_starships['crew'][df_starships['crew'] == '47,060'] = 47060
# df_starships['crew'][df_starships['crew'] == '342,953'] = 342953
# df_starships['crew'][df_starships['crew'] == '279,144'] = 279144

# #Fazendo a conversão de algumas colunas restantes para int
# df_planets['diameter'] = df_planets['diameter'].astype(int)
# df_planets['population'] = df_planets['population'].astype(np.int64)
# df_people['height'] = df_people['height'].astype(int)
# df_starships['cost_in_credits'] = df_starships['cost_in_credits'].astype(np.int64)

c:\users\macroi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\macroi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\macroi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pan

In [8]:
# Adicionando linhas com id0 para a tabela de people e planets, pois a coluna 'id_people'(chave estrangeira para tabela people) da tabela starships vai apresentar id com valores id = 0.
# E uma pessoa com id = 0, que não existe, vai precisar habitar em um planeta, por isso, também cria-se um planeta com id_planets = 0, inexistente.
df_people = adicionar_linha_nula_id0(df = df_people, tipo = 'people')
df_planets = adicionar_linha_nula_id0(df = df_planets, tipo = 'planets')

In [21]:
# Salvar os dataframes tratados em csv
save_csv(df = df_people, tipo = 'people', raw = 'no')
save_csv(df = df_starships, tipo = 'starships', raw = 'no')
save_csv(df = df_planets, tipo = 'planets', raw = 'no')

## Conexão com o BD

In [35]:
def conexao_db():
    '''
    Conecta o Python ao banco de dados.
    '''
    conexao = psycopg2.connect(host='localhost',
                               database='db_star_wars',
                               user='postgres',
                               password='admin'
                               )
    return conexao

def executar_sql(sql: str):
    '''
    Executa query SQL.
    '''
    conexao = conexao_db()
    cursor = conexao.cursor()
    cursor.execute(sql)
    conexao.commit()
    conexao.close()

def executar2_sql(sql: str, values: list):
    '''
    Executa query SQL.
    '''
    conexao = conexao_db()
    cursor = conexao.cursor()
    cursor.execute(sql, values)
    conexao.commit()
    conexao.close()
    
def executar_sql_para_select(sql: str):
    '''
    Executa query SQL para SELECT.
    '''
    conexao = conexao_db()
    resposta = psql.read_sql(sql, conexao)
    conexao.close()
    return resposta

In [23]:
# Ler os arquivos csv
df_starships_bd = pd.read_csv('starships.csv', sep = ';')
df_people_bd = pd.read_csv('people.csv', sep = ';')
df_planets_bd = pd.read_csv('planets.csv', sep = ';')

In [24]:
df_planets_bd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_planets  61 non-null     int64 
 1   name        60 non-null     object
 2   diameter    61 non-null     int64 
 3   climate     60 non-null     object
 4   gravity     60 non-null     object
 5   terrain     60 non-null     object
 6   population  61 non-null     int64 
dtypes: int64(3), object(4)
memory usage: 3.5+ KB


![](./Modelagem.png)

In [25]:
# Cria tabela planeta
executar_sql("""CREATE TABLE IF NOT EXISTS planeta (
        id_planeta SERIAL NOT NULL PRIMARY KEY, 
        nome VARCHAR,
        diametro INT,
        clima VARCHAR,
        gravidade VARCHAR,
        terreno VARCHAR,
        populacao BIGINT);""")

In [26]:
df_people_bd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id_people   83 non-null     int64  
 1   name        82 non-null     object 
 2   height      83 non-null     int64  
 3   mass        83 non-null     float64
 4   hair_color  82 non-null     object 
 5   skin_color  82 non-null     object 
 6   birth_year  82 non-null     object 
 7   gender      82 non-null     object 
 8   id_planets  83 non-null     int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 6.0+ KB


In [27]:
# Cria tabela pessoa
executar_sql("""CREATE TABLE IF NOT EXISTS pessoa (
        id_pessoa SERIAL NOT NULL PRIMARY KEY, 
        nome VARCHAR NOT NULL,
        altura INT,
        massa FLOAT,
        cor_cabelo VARCHAR,
        cor_pele VARCHAR,
        ano_nascimento VARCHAR,
        genero VARCHAR,
        id_planeta INTEGER NOT NULL,
        FOREIGN KEY (id_planeta) REFERENCES planeta(id_planeta));""")

In [29]:
df_starships_bd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_starships     36 non-null     int64  
 1   name             36 non-null     object 
 2   model            36 non-null     object 
 3   manufacturer     36 non-null     object 
 4   cost_in_credits  36 non-null     int64  
 5   length           36 non-null     float64
 6   crew             36 non-null     int64  
 7   passengers       36 non-null     int64  
 8   consumables      36 non-null     object 
 9   id_people        36 non-null     int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 2.9+ KB


In [30]:
# Cria tabela naves
executar_sql("""CREATE TABLE IF NOT EXISTS naves (
        id_naves SERIAL NOT NULL PRIMARY KEY, 
        nome VARCHAR NOT NULL,
        modelo VARCHAR NOT NULL,
        fabricante VARCHAR NOT NULL,
        custo BIGINT,
        comprimento FLOAT,
        qtd_tripulantes INT,
        qtd_passageiros INT,
        suprimento VARCHAR,
        id_pessoa INT,
        FOREIGN KEY (id_pessoa) REFERENCES pessoa(id_pessoa));""")

In [31]:
# Adiciona na tabela de planeta
for indice, linha in df_planets_bd.iterrows():
    executar2_sql(f"""INSERT INTO planeta (id_planeta, nome, diametro, clima, gravidade, terreno, populacao) 
                VALUES (%s, %s, %s, %s, %s, %s, %s);""", linha)

In [32]:
# Adiciona na tabela de pessoa
for indice, linha in df_people_bd.iterrows():
    executar2_sql(f"""INSERT INTO pessoa (id_pessoa, nome, altura, massa, cor_cabelo, cor_pele, ano_nascimento, genero, id_planeta) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);""", linha)

In [33]:
# Adiciona na tabela de naves
for indice, linha in df_starships_bd.iterrows():
    executar2_sql(f"""INSERT INTO naves (id_naves, nome, modelo, fabricante, custo, comprimento, qtd_tripulantes, qtd_passageiros, suprimento, id_pessoa) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""", linha)

## Consultando dados no Postgres

In [36]:
# Quem são os 5 personagens mais altos?
query = "select id_pessoa, nome, altura from pessoa order by altura desc limit 5;"
resposta = executar_sql_para_select(query)
resposta

,id_pessoa,nome,altura
0,57,Yarael Poof,264
1,80,Tarfful,234
2,72,Lama Su,229
3,13,Chewbacca,228
4,37,Roos Tarpals,224
